In [2]:
from selenium import webdriver as wd
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import json
from selenium.webdriver.common.action_chains import ActionChains
import random
import requests




In [3]:
def check_proxy(proxy):
    try:
        proxy = {"http": f"http://{proxy}", "https": f"http://{proxy}"}
        response = requests.get("https://httpbin.org/ip", proxies=proxy, timeout=5)
        if response.status_code == 200:
            print(f"Прокси {proxy} работает. Ответ: {response.json()}")
            return True
    except requests.exceptions.RequestException:
        print(f"Прокси {proxy} не работает.")
    return False

In [4]:
def change_proxy(url): #ссылка для получения proxy
    working_proxies = []
    response = requests.get(url)
    response = response.text
    proxies = response.split('\n')
    working_proxies = []
    for proxy in proxies:
        if check_proxy(proxy):
            working_proxies.append(proxy)
    proxy_ip_port = random.choice(working_proxies)
    
    chrome_options = wd.ChromeOptions()
    chrome_options.add_argument('--proxy-server=%s' % proxy_ip_port)
    return chrome_options

In [5]:
def save_data(data, filename):
    """Сохраняет данные в JSON-файл."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Данные сохранены в файл {filename}")
    except Exception as e:
        print(f"Ошибка при сохранении данных в файл: {e}")

In [8]:
def data_pars(url): #ссылка на авито-резюме
    driver = wd.Chrome()
    driver.get(url)
    
    # Ожидаем загрузки элементов на странице
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'items-items-Iy89l')))
    
    data = {}
    
    # Переходим по страницам
    for i in range(20):
        try:
            current_url = driver.current_url
            elem = driver.find_element(By.ID, 'bx_serp-item-list')
            button0 = elem.find_elements(By.CLASS_NAME, 'iva-item-sliderLink-Fvfau')

            # Находим все ссылки на элементах
           
            time.sleep(10)
            #button0 = elem.find_elements(By.TAG_NAME, 'a')
            links = []
            for button in button0:
                links.append(button.get_attribute('href'))
            links = list(set(links))
            print(len(links))
            for button in button0:
                driver.execute_script("arguments[0].scrollIntoView();", button)
                button.click()

                windows = driver.window_handles  # получаем список окон

                for scr in windows[1:]: 
                    driver.switch_to.window(scr)# переключаемся на все новые окна
                    
                        # Ждем, пока загрузится необходимый элемент на странице
                            #WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div[3]/div[1]/div/div[2]/div[3]/div/div[1]/div[2]/div[5]/div/article/p/span[1]')))
                    time.sleep(10)
                    # Собираем данные
                    pars_data = {}
                    # Парсим номер
                    item_element = driver.find_element(By.CSS_SELECTOR, '[data-marker="item-view/item-id"]')
                    pars_data.update({'num': item_element.text})
                    try:
                        education = driver.find_elements(By.CLASS_NAME, 'style-serviceEducationWrapper-xvAfY')
                        for e in education:
                            pars_data.update({'education': item_element.text})
                    except:
                        pars_data.update({'education': 'Не указано'})
                        pass


                    # Парсим таблицу
                    table = driver.find_elements(By.CLASS_NAME, 'style-item-description-pL_gy')
                    for t in table:
                        pars_data.update({'description': t.text})
                    try:
                        rate = driver.find_elements(By.CSS_SELECTOR,'span.style-seller-info-rating-score-C0y96')
                        for r in rate:
                            pars_data.update({'rate': r.text})

                        sum_review = driver.find_elements(By.CSS_SELECTOR, 'a[data-marker="rating-caption/rating"]')
                        for sum_r in sum_review:
                            pars_data.update({'sum_review': sum_r.text})    
                    except:
                        pars_data.update({'rate': 'Оценок нет'})
                        pars_data.update({'sum_revie': 'Оценок нет'})
                        pass

                    try:
                        xpath = '//*[@id="app"]/div/div[3]/div[1]/div/div[2]/div[3]/div/div[2]/div[1]/div/div/div[4]/div[1]/div/div/div/div[1]/div/div[1]/div/div[2]/a'
                        button_review = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-marker="rating-caption/rating"]')))
                        button_review.click()
                        time.sleep(10)
                        elements = driver.find_elements(By.CSS_SELECTOR, 'p[data-marker^="review("]')
                        texts = [el.text.strip() for el in elements]
                        pars_data.update({'review': texts})
                        #rating_review = driver.find_element(By.XPATH, '/html/body/div[2]/div[11]/div/div[2]/div/div/div/div/div/div[1]/div[2]/div/div')
                        #for r in rating_review:
                                #pars_data.update({'rating': r.text})
                    except:
                        pars_data.update({'review': 'Нет отзывов'}) 
                        pass
                
                current_index = (button0.index(button)//2) + i*50
                data[current_index] = pars_data
                time.sleep(5)
    

                # Закрываем окно и возвращаемся к основному
                driver.close()
                driver.switch_to.window(windows[0])
            
       

            # Переход на следующую страницу
            try:
                next_button = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[data-marker="pagination-button/nextPage"]')))
                next_page_url = next_button.get_attribute('href')
                # Переходим на следующую страницу
                driver.get(next_page_url)
            
            except Exception as e:
                print("Ошибка при переходе на следующую страницу:", e)
                break
        except Exception as e:
            print(f"Ошибка на странице {i + 1}: {e}")
            # Сохраняем данные в случае ошибки
            save_data(data, f'{i}_page__.json')
            driver.close()
            driver.switch_to.window(windows[0])
            continue
        
    driver.quit()  # Закрываем браузер
    return data


In [9]:
data2 = data_pars('https://www.avito.ru/all/predlozheniya_uslug/krasota/brovi_resnicy-ASgBAgICAkSYC6qfAaIrrOSKAw?f=ASgBAgICA0SYC6qfAaIrrOSKA566FOC~jwM&p=15&s=104')

50
50
50
50
50
Ошибка на странице 5: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-marker="item-view/item-id"]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104a0aa54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x0000000104a02cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x000000010454e864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000104595410 cxxbridge1$string$len + 382736
4   chromedriver                        0x00000001045d6480 cxxbridge1$string$len + 649088
5   chromedriver                        0x00000001045897ec cxxbridge1$string$len + 334572
6   chromedriver                        0x00000001049cfccc cxxbridge1$str$ptr + 2562928
7   chromedriver 

In [10]:
save_data(data2, 'data20.json')

Данные сохранены в файл data20.json
